In [1]:
# Setting New Working Directory.

# First Import Python's os Module.
import os

# Print Current Working Directory.
print("Current Working Directory is:",os.getcwd())

# Change the Current Working Directory using os.chdir(path)
os.chdir("C:/Users/gabaj/Downloads/Capstone python/Capstone python KNN")
print("New Working Directory is:",os.getcwd())

Current Working Directory is: C:\Users\gabaj\2 Capstone
New Working Directory is: C:\Users\gabaj\Downloads\Capstone python\Capstone python KNN


In [2]:
# Suppress Warnings
import warnings
warnings.filterwarnings('ignore')

In [3]:
## Import required packages
%matplotlib inline

from pathlib import Path

import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
import matplotlib.pylab as plt


# Data Import & Partioning

In [4]:
cloth_df = pd.read_csv('clothsize.csv')
cloth_df['Number'] = cloth_df.index + 1
cloth_df.head(9)

height  weight  size  Number
0  162.56      64    XL       1
1  167.64      49     S       2
2  167.64      65     L       3
3  170.18      57     M       4
4  167.64      56     M       5
5  167.64      54     S       6
6  167.64      72  XXXL       7
7  165.10      72     M       8
8  167.64      54   XXS       9

In [5]:
trainData, validData = train_test_split(cloth_df, test_size=0.05, random_state=26)
print(trainData.shape, validData.shape)
newSize = pd.DataFrame([{'height': 175, 'weight': 62}])
newSize

(531, 4) (28, 4)


height  weight
0     175      62

# Normalising Data & Finding Nearest Numbers

In [6]:
scaler = preprocessing.StandardScaler()
scaler.fit(trainData[['height', 'weight']])  # Note the use of an array of column names

# Transform the full dataset
clothNorm = pd.concat([pd.DataFrame(scaler.transform(cloth_df[['height', 'weight']]), 
                                    columns=['zheight', 'zweight']),
                       cloth_df[['size', 'Number']]], axis=1)
trainNorm = clothNorm.iloc[trainData.index]
validNorm = clothNorm.iloc[validData.index]
newSizeNorm = pd.DataFrame(scaler.transform(newSize), columns=['zheight', 'zweight'])

Use k-nearest neighbour

In [7]:
knn = NearestNeighbors(n_neighbors=3)
knn.fit(trainNorm[['zheight', 'zweight']])
distances, indices = knn.kneighbors(newSizeNorm)
print(trainNorm.iloc[indices[0], :])  # indices is a list of lists, we are only interested in the first element

      zheight   zweight size  Number
318  1.442372 -0.037498    L     319
250  1.442372 -0.037498    L     251
214  1.442372 -0.037498    L     215


# Choosing appropriate K
Initialize a data frame with two columns: k and accuracy

In [8]:
train_X = trainNorm[['zheight', 'zweight']]
train_y = trainNorm['size']
valid_X = validNorm[['zheight', 'zweight']]
valid_y = validNorm['size']

# Train a classifier for different values of k
results = []
for k in range(1, 532):
    knn = KNeighborsClassifier(n_neighbors=k).fit(train_X, train_y)
    results.append({
        'k': k,
        'accuracy': accuracy_score(valid_y, knn.predict(valid_X))
    })

# Convert results to a pandas data frame
results = pd.DataFrame(results)
print(results)

       k  accuracy
0      1  0.464286
1      2  0.357143
2      3  0.428571
3      4  0.535714
4      5  0.500000
..   ...       ...
526  527  0.071429
527  528  0.071429
528  529  0.071429
529  530  0.071429
530  531  0.071429

[531 rows x 2 columns]


In [9]:
#see maximum accuracy
max(results.accuracy)

0.5714285714285714

In [10]:
#look for the value of 0.571428 from results and match with k
print(results[results.accuracy == results.accuracy.max()])

     k  accuracy
6    7  0.571429
8    9  0.571429
13  14  0.571429
14  15  0.571429


# KNN Model with Complete Dataset

In [11]:
# Retrain with full dataset
cloth_X = clothNorm[['zheight', 'zweight']]
cloth_y = clothNorm['size']
knn = KNeighborsClassifier(n_neighbors=7).fit(cloth_X, cloth_y)
distances, indices = knn.kneighbors(newSizeNorm)
print(knn.predict(newSizeNorm))
print('Distances',distances)
print('Indices', indices)
print(clothNorm.iloc[indices[0], :])

['L']
Distances [[0.12494147 0.12494147 0.12494147 0.12494147 0.12494147 0.12494147
  0.24031705]]
Indices [[413 250 318 214 317 249 319]]
      zheight   zweight size  Number
413  1.442372  0.199544   XL     414
250  1.442372 -0.037498    L     251
318  1.442372 -0.037498    L     319
214  1.442372 -0.037498    L     215
317  1.442372  0.199544   XL     318
249  1.442372  0.199544    L     250
319  1.442372  0.318066    L     320
